In [1]:
#Usual imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#my modules
import wrangle as wr
import evaluate as ev
import explore as exp
from env import get_db_url

#sklearn imports
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.feature_selection import f_regression, SelectKBest, RFE
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor

#extras
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)




Getting a fresh copy from SQL db...
Saving to CSV...


In [2]:
prop_vals = wr.get_prop_vals()
prop_vals.info()
prop_vals.head()

Getting a fresh copy from SQL database...
Copying to CSV...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
 7   propertylandusetypeid         float64
dtypes: float64(8)
memory usage: 182.2 MB


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
0,0.0,0.0,NaN,9.0,NaN,NaN,6037.0,269.0
1,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0
2,0.0,0.0,73026.0,1434941.0,1959.0,20800.37,6037.0,47.0
3,0.0,0.0,5068.0,1174475.0,1948.0,14557.57,6037.0,47.0
4,0.0,0.0,1776.0,440101.0,1947.0,5725.17,6037.0,31.0
